### Iterables

Arrays are iterable. But there are many other built-in objects, that are iterable as well. For instance, strings are iterable also. If an object represents a collection (list, set) of something, the `for .. of` is a great syntax to loop over it, so we'll see how to make it work.

#### `Symbol.iterator`

Suppose we have an object, that is not an array, but looks suitable for `for .. of`. Like a `range` object that represents an interval of numbers - 

```js
let range = {
from : 1
to : 5
};
```
To make the `range` iterable, we need to add a method to the object named `Symbol.iterator` (a special built-in symbol just for that).

 - When `for ..  of` starts, it calls that method once. The method must return an iterator - an object with the method `next`.
 - Onward, `for .. of` works only with that returned object. 
 - When `for .. of` wants the next value, it calls `next()` on that object.
 - The result of `next()` must have the form `{done: Boolean, value: any}`, where `done = true` means that iteration is finished, otherwise `value` must be the new value.
 
See below for implementation for `range` -

In [1]:
let range = {
    from: 1,
    to:5
};

range[Symbol.iterator] = function(){
    return {
        current: this.from,
        last: this.to,
        next(){
            if (this.current <= this.last){
                return {done:false, value: this.current++};
            } else {
                return {done: true};
            }
        }
    };
};

for (let num of range){
    console.log(num);
}

1
2
3
4
5


Please note the core feature of iterables: an important separation of concerns:

 - The `range` itself does not have the `next()` method.
 - Instead, another object, a so called 'iterator' is created by the call to `range[Symbol.iterator]()` and it handles the whole iteration. 
 
So, the iterator object is separate from the object it iterates over.

Technically, we may merge them and use `range` itself as the iterator to make the code simpler -

In [3]:
let range1 = {
    from: 1,
    to: 5,
    [Symbol.iterator](){
        this.current = this.from;
        return this;
    },
    next(){
        if (this.current <= this.to){
            return {done: false, value: this.current++};
        } else {
            return {done: true}
        }
    }
};

for (let num of range1){
    console.log(num);
}

1
2
3
4
5


The downside with above approach is that now it's impossible to have two `for .. of` loops running over the object simultaneously: they'll share the iteration state, because there's only one iterator - the object itself. However, we rarely encounter this situation and there are workarounds for that.

**Infinite iterators**

Infinite are iterators are also possible. Above, we can make `range` infinite by setting `range.to =  Infinity`.

#### String is iterable

For a string, `for..of` loops over its characters:

In [4]:
for (let char of "test"){
    console.log(char)
}

t
e
s
t


And it works correctly with surrogate pairs - 

In [18]:
uuu = 'ÚÛÜ'
for (let char of uuu){
    console.log(char)
}

Ú
Û
Ü


#### Calling an iterator explicitly

Normally, internals of iterables are hidden from the external code. There's a `for ..  of` loop, that works, that's all it needs to know. But to understand things a little deeper let's see how to create an iterator explicitly. 

We'll iterate over a string the same way as `for .. of`, but with direct calls. This code gets a string iterator and calls it 'manually' -

In [5]:
let str = 'Hello';
let iterator = str[Symbol.iterator]();

while(true){
    let result = iterator.next()
    if (result.done) break;
    console.log(result.value)
}


H
e
l
l
o


#### Iterables and array-likes

There are two official terms that look similar, but are very different. 

 - Iterables are objects that implement the `Symbol.iterator` method, as described above.
 - Array-likes are objects that have indexes and `length`, so they look like arrays
 
Naturally, these properties can combine. For instance, strings are both iterable (`for ..of` works on them) and array-like (they have indexes and `length`).

But an iterable may be not array-like and vice versa. Above `range` is iterable but not array-like as it doesn't have indexed properties and `length`.

Below is an example of the object which is array-like but not iterable -

In [6]:
let arrayLike = {
    0: 'Hello',
    1: 'World',
    length : 2
};

//error
for (let item of arrayLike){
    console.log(item)
}

TypeError: arrayLike is not iterable

#### `Array.from`

There's a universal method `Array.from` that brings them together. It takes an iterable or array-like value and makes a 'real' `Array` from it. Then we can call array methods on it.  

In [7]:
arrayLike

{ '0': 'Hello', '1': 'World', length: 2 }

In [8]:
let arr = Array.from(arrayLike);
arr.pop()

'World'

The same happens for an iterable - 


In [9]:
let new_array = Array.from(range);
console.log(new_array)

[ 1, 2, 3, 4, 5 ]


The full syntax for `Array.from` allows to provide an option 'mapping' function - 

```js
Array.from(obj[, mapFn, thisArg])
```

The second argument `mapFn` should be the function to apply to each element before adding to the array and `thisArg` allows to set `this` for it.

In [10]:
let abc = Array.from(range, num=>num*num);
abc

[ 1, 4, 9, 16, 25 ]

In [12]:
let foo = 'ÚÛÜ';
let bar = Array.from(st);
bar[0]

'Ú'

In [13]:
bar.length

3

Unlike `str.split`, it relies on the iterable nature of the string and so, just like `for .. of`, correctly works with surrogate pairs.

Technically here it does the same as -

In [14]:
foo

'ÚÛÜ'

In [15]:
let list = [];
for (let char of foo){
    list.push(char)
}
list

[ 'Ú', 'Û', 'Ü' ]

We can even build surrogate-aware `slice` on it - 

In [16]:
function slice(str, start, end){
    return Array.from(str).slice(start, end).join('');
}

slice(foo,1,2)

'Û'